In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import yfinance as yf
import pandas as pd
import math
from scipy.stats import norm
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Bidirectional, Dropout, Conv1D, MaxPooling1D, Flatten, Attention
from tensorflow.keras.models import Model
from ta.volume import MFIIndicator
from ta.utils import dropna
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import requests
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [2]:
# My function
def make_model():
    # 1. Input layer for stock data (sequence_length, features)
    inp = Input(shape=(128, 9))  # Example: 128 timesteps, 10 features (adjust as needed)

    # 2. Bidirectional LSTM layers for sequence learning
    x = Bidirectional(LSTM(128, return_sequences=True))(inp)
    x = Bidirectional(LSTM(64, return_sequences=True))(x)

    # 3. Dropout to prevent overfitting
    x = Dropout(0.2)(x)

    # 4. 1D Convolutional layer for feature extraction
    x = Conv1D(filters=64, kernel_size=3, activation="relu", padding="same")(x)

    # 5. MaxPooling to downsample
    x = MaxPooling1D(pool_size=2)(x)

    # 6. Flatten layer to transition to fully connected layers
    x = Flatten()(x)

    # 7. Fully connected layers for high-level learning
    x = Dense(128, activation="relu")(x)
    x = Dense(64, activation="relu")(x)

    # 8. Output layer - Single neuron with tanh activation to produce values between -1 and 1
    output = Dense(1, activation="tanh")(x)

    # 9. Compile model with Adam optimizer
    model = Model(inputs=inp, outputs=output)
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])

    return model

# Create the model
model = make_model()
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 128, 9)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 128, 256)       │       141,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 128, 128)       │       164,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 128, 64)        │        24,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 64, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       524,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 863,041 (3.29 MB)

 Trainable params: 863,041 (3.29 MB)

 Non-trainable params: 0 (0.00 B)

In [3]:
def get_stock_data(ticker, start='2024-01-01', end='2025-03-24', buffer_days=200):
    # Extend the start date backward by the buffer_days
    extended_start = pd.to_datetime(start) - pd.Timedelta(days=buffer_days)
    stock = yf.download(ticker, start=extended_start.strftime('%Y-%m-%d'), end=end)
    return stock

In [4]:
apple = get_stock_data('AAPL')

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
